# Tasks
Organizing the COCO data:

- Create a class that organizes all of the COCO data. It might store the following
    - All the image IDs
    - All the caption IDs
    - Various mappings between image/caption IDs, and associating caption-IDs with captions
        - `image-ID -> [cap-ID-1, cap-ID-2, ...]`
        - `caption-ID -> image-ID`
        - `caption-ID -> caption (e.g. 24 -> "two dogs on the grass")`

# Loading Data

In [2]:
from cogworks_data.language import get_data_path

from pathlib import Path
import json

# load COCO metadata
filename = get_data_path("captions_train2014.json")
with Path(filename).open() as f:
    coco_data = json.load(f)

# >>> len(coco_data["images"])  # number of images
# 82783

# >>> coco_data["images"][0]
# {'license': 5,
#  'file_name': 'COCO_train2014_000000057870.jpg',
#  'coco_url': 'http://images.cocodataset.org/train2014/COCO_train2014_000000057870.jpg',
#  'height': 480,
#  'width': 640,
#  'date_captured': '2013-11-14 16:28:13',
#  'flickr_url': 'http://farm4.staticflickr.com/3153/2970773875_164f0c0b83_z.jpg',
#  'id': 57870}

In [3]:
from gensim.models import KeyedVectors
filename = "glove.6B.200d.txt.w2v"

# this takes a while to load -- keep this in mind when designing your capstone project
glove = KeyedVectors.load_word2vec_format(get_data_path(filename), binary=False)

In [4]:
import pickle
with Path(get_data_path('resnet18_features.pkl')).open('rb') as f:
    resnet18_features = pickle.load(f)

# COCO Data Class

In [5]:
c = coco_data["annotations"]
# print (c[:10])

i = coco_data["images"]
print(type(i))

print(type(coco_data))
print(type(resnet18_features))

<class 'list'>
<class 'dict'>
<class 'dict'>


In [6]:
class CocoDataManager:
    """
    Attributes
    ----------
    images : list
        list of image dictionaries
    captions : list
        list of caption dictionaries
    imageID_to_captionID : dictionary {int : int}
        dictionary that stores image IDs as the key and caption IDs as the value
    captionID_to_imgID : dictionary {int : int}
        dictionary that stores captions IDs as the key and image IDs as the value
    captionID_to_caption : dictionary {int : str}
        dictionary that stores captions IDs as the key and a caption as the value
    caption_to_captionID : dictionary {str : int}
        dictionary that stores captions as the key and a caption IDs as the value
    imageID_to_url : dictionary {int : str}
        dictionary that stores image IDs as the key and a coco URL as the value
    imageIDs : list
        unsorted list of all image IDs
    captionID_to_captionEmbedding : dictionary {int : np.array(200,)}
        dictionary that stores caption IDs as the key and a caption embedding as the value
    
    Methods
    -------
    getCaptionIDs(self, img_id)
        Gives a list of caption IDs given an image ID
    getUrl(self, img_id)
        Gives a coco url given an image ID
    """
    
    def __init__ (self, coco_data):
        from OmniCog import query_embed
        """
        Initialize a CocoDataManager class instance

        Parameters
        ----------
        coco_data : dictionary
            The loaded json file as a dictionary
        """
        
        self.unprocessed_images = coco_data["images"]
        self.captions = coco_data["annotations"]
        
        self.images = []
        
        # removing captions without ResNet descriptor
        for i in range(len(self.unprocessed_images)):
            if (self.unprocessed_images[i]["id"] in resnet18_features):
                self.images.append(self.unprocessed_images[i])
        
        
        self.imageID_to_captionID = {}
        self.captionID_to_imgID = {}
        
        self.captionID_to_caption = {}
        self.caption_to_captionID = {}
        
        self.imageID_to_url = {}
        
        self.imageIDs = [i["id"] for i in self.images]
        
        self.captionID_to_captionEmbedding = {}
        
        for c in self.captions:
            cap_id = c["id"]
            img_id = c["image_id"]
            cap = c["caption"]
            
            # adding data to the imageID to captionID dictionary
            if (img_id in self.imageID_to_captionID):
                self.imageID_to_captionID[img_id].append(cap_id)
            else:
                self.imageID_to_captionID[img_id] = [cap_id]
            
            # adding data to the captionID to imageID dictionary
            self.captionID_to_imgID[cap_id] = img_id
            
            # adding data to the captionID to caption dictionary
            self.captionID_to_caption[cap_id] = cap
            
            # adding data to the captionID to caption embedding dictionary
            self.captionID_to_captionEmbedding[cap_id] = query_embed(cap)
            
        # adding data to the caption to captionID dictionary
        self.caption_to_captionID = {value:key for key, value in self.captionID_to_caption.items()}
        
        for img in self.images:
            img_id = img["id"]
            img_url = img["coco_url"]
            img_height = img["height"]
            img_width = img["width"]
        
            # adding data to the imageID to url dictionary
            self.imageID_to_url[img_id] = img_url
                
    def getCaptionIDs (self, img_id):
        """
        Gives a list of caption IDs given an image ID

        Parameters
        ----------
        img_id : int
            Id of the image to be accessed

        Returns
        -------
        list
            List of length 4 that contains the caption ids
        """
        return self.imageID_to_captionID[img_id]
    
    def getUrl (self, img_id):
        """
        Gives a coco url given an image ID

        Parameters
        ----------
        img_id : int
            Id of the image to be accessed

        Returns
        -------
        String
            The coco URL used to access/download an image
        """
        return self.imageID_to_url[img_id]


In [8]:
manager = CocoDataManager(coco_data)


NameError: name 'np' is not defined

In [ ]:
len(resnet18_features) < len(manager.images)

In [ ]:
len(resnet18_features)

In [ ]:
len(manager.images)